In [ ]:
import os
import ast
import json
import time
import base64
import requests
import threading
import websocket
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime, timezone

from datetime import datetime, timedelta
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding


load_dotenv()


# -------------------------------------------------
# CAPITAL.COM CLIENT
# -------------------------------------------------
class CapitalClient:
    CANDLES_PER_MINUTE = {
        "MINUTE": 1,
        "MINUTE_5": 1 / 5,
        "MINUTE_15": 1 / 15,
        "MINUTE_30": 1 / 30,
        "HOUR": 1 / 60,
        "HOUR_4": 1 / 240,
        "DAY": 1 / 1440,
        "WEEK": 1 / 10080,
    }

    CHUNK_MINUTES = {
        "MINUTE": 600,
        "MINUTE_5": 3000,
        "MINUTE_15": 6000,
        "MINUTE_30": 12000,
        "HOUR": 43200,
        "HOUR_4": 172800,
        "DAY": 525600,
        "WEEK": 1048320,
    }

    def __init__(self, api_key: str, identifier: str, password: str):
        self.api_key = api_key
        self.identifier = identifier
        self.password = password
        self.base_url = os.getenv("CAPITAL_BASE_URL")

        # Session management
        self.cst = None
        self.security_token = None
        self.session_expiry = None
        self._session_lock = threading.Lock()

    # ----------------------------------
    # SESSION MANAGEMENT
    # ----------------------------------
    def _is_session_valid(self) -> bool:
        """Check if the current session is still valid."""
        if not self.cst or not self.security_token:
            return False

        # If we don't have expiry time, assume it's valid
        if not self.session_expiry:
            return True

        # Add a buffer of 5 minutes before actual expiry
        return datetime.now(timezone.utc) < (self.session_expiry - timedelta(minutes=5))

    def _renew_session(self):
        """Renew the session by logging in again."""
        with self._session_lock:
            print("🔄 Renewing session...")
            self.login()

    def _ensure_valid_session(self):
        """Ensure we have a valid session, renew if needed."""
        if not self._is_session_valid():
            self._renew_session()

    # ----------------------------------
    # AUTH
    # ----------------------------------
    def login(self):
        """Login and store session expiry information."""
        r = requests.get(
            f"{self.base_url}/session/encryptionKey",
            headers={"X-CAP-API-KEY": self.api_key},
        )
        r.raise_for_status()

        encryption_key = r.json()["encryptionKey"]
        timestamp = r.json()["timeStamp"]

        message = f"{self.password}|{timestamp}".encode()
        message_b64 = base64.b64encode(message)

        public_key = serialization.load_der_public_key(base64.b64decode(encryption_key))
        encrypted = public_key.encrypt(message_b64, padding.PKCS1v15())
        encrypted_password = base64.b64encode(encrypted).decode()

        r = requests.post(
            f"{self.base_url}/session",
            headers={
                "X-CAP-API-KEY": self.api_key,
                "Content-Type": "application/json",
            },
            json={
                "identifier": self.identifier,
                "password": encrypted_password,
                "encryptedPassword": True,
            },
        )
        r.raise_for_status()

        self.cst = r.headers["CST"]
        self.security_token = r.headers["X-SECURITY-TOKEN"]

        # Try to parse expiry from response if available
        try:
            account_info = r.json().get("accountInfo", {})
            # Capital.com sessions typically last 24 hours
            self.session_expiry = datetime.now(timezone.utc) + timedelta(
                hours=23
            )  # Conservative estimate
        except:
            # If we can't parse expiry, set to 23 hours from now
            self.session_expiry = datetime.now(timezone.utc) + timedelta(hours=23)

        print("✅ Logged in successfully")

    @property
    def headers(self):
        """Get headers with automatic session renewal."""
        self._ensure_valid_session()

        if not self.cst or not self.security_token:
            raise RuntimeError("Call login() first")

        return {
            "X-CAP-API-KEY": self.api_key,
            "CST": self.cst,
            "X-SECURITY-TOKEN": self.security_token,
        }

    # ----------------------------------
    # WEB SOCKET SESSION MANAGEMENT
    # ----------------------------------
    def get_websocket_headers(self):
        """Get headers specifically for WebSocket connections."""
        self._ensure_valid_session()
        return {
            "X-CAP-API-KEY": self.api_key,
            "CST": self.cst,
            "X-SECURITY-TOKEN": self.security_token,
        }

    # ----------------------------------
    # RAW PRICE CALL (UPDATED WITH TIMEZONE HANDLING)
    # ----------------------------------
    def _fetch_prices(
        self,
        epic: str,
        resolution: str,
        start: datetime,
        end: datetime,
    ) -> pd.DataFrame:

        minutes = (end - start).total_seconds() / 60
        max_points = int(minutes * self.CANDLES_PER_MINUTE[resolution]) + 5

        # Convert to UTC and remove timezone info for Capital.com API
        if start.tzinfo is not None:
            start = start.astimezone(timezone.utc).replace(tzinfo=None)

        if end.tzinfo is not None:
            end = end.astimezone(timezone.utc).replace(tzinfo=None)

        # Format as simple ISO string without timezone
        start_iso = start.strftime("%Y-%m-%dT%H:%M:%S")
        end_iso = end.strftime("%Y-%m-%dT%H:%M:%S")

        params = {
            "resolution": resolution,
            "from": start_iso,
            "to": end_iso,
            "max": max_points,
        }

        r = requests.get(
            f"{self.base_url}/prices/{epic}",
            headers=self.headers,
            params=params,
        )
        r.raise_for_status()

        prices = r.json().get("prices", [])
        if not prices:
            return pd.DataFrame()

        df = pd.DataFrame(prices)

        # Convert to UTC timezone-aware datetime
        df["timestamp"] = pd.to_datetime(df["snapshotTime"]).dt.tz_localize(None)

        # For 15-minute candles, check if we need to round to the nearest 15 minutes
        if resolution == "MINUTE_15":
            # Round down to the nearest 15 minutes
            df["timestamp"] = df["timestamp"].dt.floor("15min")

        df = df.sort_values("timestamp")

        return df[["timestamp", "openPrice", "highPrice", "lowPrice", "closePrice"]]

    # ----------------------------------
    # SMART HISTORICAL DOWNLOAD (UPDATED)
    # ----------------------------------
    def get_historical_prices(
        self,
        epic: str,
        resolution: str,
        from_date: str,
        to_date: str,
        timezone_offset: int = 0,  # Timezone offset in hours (e.g., 5.5 for IST)
    ) -> pd.DataFrame:
        """
        Get historical prices with timezone handling.
        """

        # Parse dates (handle both date-only and datetime strings)
        try:
            start_dt = datetime.fromisoformat(from_date)
        except ValueError:
            # If only date is provided, add time
            start_dt = datetime.strptime(from_date, "%Y-%m-%d")

        try:
            end_dt = datetime.fromisoformat(to_date)
        except ValueError:
            # If only date is provided, add end of day
            end_dt = datetime.strptime(to_date, "%Y-%m-%d")
            end_dt = end_dt.replace(hour=23, minute=59, second=59, microsecond=999999)

        # Apply timezone offset if needed
        if timezone_offset != 0:
            start_dt = start_dt + timedelta(hours=timezone_offset)
            end_dt = end_dt + timedelta(hours=timezone_offset)
            print(f"📅 Applied {timezone_offset:+} hour timezone offset")

        delta = timedelta(minutes=self.CHUNK_MINUTES[resolution])

        all_chunks = []
        current = start_dt

        print(f"\n📥 Downloading {epic} {resolution} candles")
        print(
            f"   Date range: {start_dt.strftime('%Y-%m-%d %H:%M:%S')} to {end_dt.strftime('%Y-%m-%d %H:%M:%S')}"
        )

        while current < end_dt:
            chunk_end = min(current + delta, end_dt)

            print(
                f"  {current.strftime('%Y-%m-%d %H:%M:%S')} → {chunk_end.strftime('%Y-%m-%d %H:%M:%S')}"
            )

            df = self._fetch_prices(
                epic=epic,
                resolution=resolution,
                start=current,
                end=chunk_end,
            )

            if not df.empty:
                all_chunks.append(df)

            current = chunk_end

        if not all_chunks:
            return pd.DataFrame()

        df = (
            pd.concat(all_chunks)
            .drop_duplicates("timestamp")
            .sort_values("timestamp")
            .reset_index(drop=True)
        )

        # Apply reverse timezone offset to display in local time
        if timezone_offset != 0:
            df["timestamp"] = df["timestamp"] - pd.Timedelta(hours=timezone_offset)

        print(
            f"✅ Done: {len(df):,} candles "
            f"({df.timestamp.min().strftime('%Y-%m-%d %H:%M:%S')} → {df.timestamp.max().strftime('%Y-%m-%d %H:%M:%S')})"
        )

        # Check for missing candles
        if len(df) > 1:
            expected_freq = "15min" if resolution == "MINUTE_15" else "1D"
            full_range = pd.date_range(
                start=df.timestamp.min(), end=df.timestamp.max(), freq=expected_freq
            )
            missing = set(full_range) - set(df.timestamp)
            if missing:
                print(f"⚠️  Missing {len(missing)} candles at: {list(missing)[:5]}...")

        return df

    # ----------------------------------
    # ACCOUNTS (BALANCE / MARGIN / P&L)
    # ----------------------------------
    def get_accounts(self) -> pd.DataFrame:
        """
        Fetch all trading accounts with balances and margin info.
        """
        print(self.base_url)
        r = requests.get(
            f"{self.base_url}/accounts",
            headers=self.headers,
        )
        r.raise_for_status()

        accounts = r.json().get("accounts", [])
        if not accounts:
            return pd.DataFrame()

        df = pd.json_normalize(accounts)

        # Optional: cleaner column names
        df = df.rename(
            columns={
                "balance.balance": "balance",
                "balance.available": "available",
                "balance.deposit": "deposit",
                "balance.profitLoss": "profit_loss",
            }
        )

        return df

    # ----------------------------------
    # MARKET SEARCH
    # ----------------------------------
    def search_markets(self, search_term=""):
        r = requests.get(
            f"{self.base_url}/markets",
            headers=self.headers,
            params={"searchTerm": search_term, "limit": 20},
        )
        r.raise_for_status()
        return r.json().get("markets", [])

        # ----------------------------------

    # WORKING ORDERS (LIMIT/STOP ORDERS)
    # ----------------------------------
    def create_working_order(
        self,
        epic: str,
        direction: str,
        size: float,
        level: float,
        order_type: str,
        good_till_date: str = None,
        guaranteed_stop: bool = False,
        trailing_stop: bool = False,
        stop_level: float = None,
        stop_distance: float = None,
        stop_amount: float = None,
        profit_level: float = None,
        profit_distance: float = None,
        profit_amount: float = None,
        deal_reference: str = None,
    ) -> dict:
        """Create a limit or stop order (working order)."""

        # Validate required parameters
        if direction not in ["BUY", "SELL"]:
            raise ValueError("direction must be either 'BUY' or 'SELL'")

        if size <= 0:
            raise ValueError("size must be greater than 0")

        if level <= 0:
            raise ValueError("level must be greater than 0")

        if order_type not in ["LIMIT", "STOP"]:
            raise ValueError("order_type must be either 'LIMIT' or 'STOP'")

        # Validate stop parameters
        if guaranteed_stop and trailing_stop:
            raise ValueError("Cannot set both guaranteedStop and trailingStop to True")

        if guaranteed_stop:
            if not any([stop_level, stop_distance, stop_amount]):
                raise ValueError(
                    "If guaranteedStop=True, must provide stopLevel, stopDistance, or stopAmount"
                )

        if trailing_stop:
            if stop_distance is None:
                raise ValueError("If trailingStop=True, must provide stopDistance")

        # Validate good_till_date format if provided
        if good_till_date:
            try:
                datetime.fromisoformat(good_till_date.replace("Z", ""))
            except ValueError:
                raise ValueError("good_till_date must be in format YYYY-MM-DDTHH:MM:SS")

        # Prepare request body
        body = {
            "epic": epic,
            "direction": direction,
            "size": size,
            "level": level,
            "type": order_type,
            "guaranteedStop": guaranteed_stop,
            "trailingStop": trailing_stop,
        }

        # Add optional parameters if provided
        if good_till_date is not None:
            body["goodTillDate"] = good_till_date

        if stop_level is not None:
            body["stopLevel"] = stop_level

        if stop_distance is not None:
            body["stopDistance"] = stop_distance

        if stop_amount is not None:
            body["stopAmount"] = stop_amount

        if profit_level is not None:
            body["profitLevel"] = profit_level

        if profit_distance is not None:
            body["profitDistance"] = profit_distance

        if profit_amount is not None:
            body["profitAmount"] = profit_amount

        if deal_reference is not None:
            body["dealReference"] = deal_reference

        # Make the API request
        r = requests.post(
            f"{self.base_url}/workingorders", headers=self.headers, json=body
        )
        r.raise_for_status()

        response = r.json()

        # Add convenience fields
        if "dealReference" in response:
            response["order_id"] = response["dealReference"].replace("o_", "")
            response["dealId"] = response["order_id"]

        order_type_desc = "Limit" if order_type == "LIMIT" else "Stop"
        print(
            f"✅ {order_type_desc} order created: {direction} {size} {epic} @ {level}"
        )
        if "dealReference" in response:
            print(f"   Deal Reference: {response['dealReference']}")

        return response

    # ----------------------------------
    # GET ALL WORKING ORDERS
    # ----------------------------------
    def get_working_orders(self) -> pd.DataFrame:
        """
        Get all pending working orders (limit/stop orders).

        """
        r = requests.get(f"{self.base_url}/workingorders", headers=self.headers)
        r.raise_for_status()

        orders = r.json().get("workingOrders", [])
        if not orders:
            return pd.DataFrame()

        df = pd.json_normalize(orders)

        # Optional: Clean up column names
        df = df.rename(columns=lambda x: x.replace(".", "_"))

        # Convert timestamp columns if they exist
        timestamp_cols = ["createdDate", "goodTillDate"]
        for col in timestamp_cols:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col])

        return df

    # ----------------------------------
    # GET SPECIFIC WORKING ORDER
    # ----------------------------------
    def get_working_order(self, deal_id: str) -> dict:
        """
        Get details of a specific working order.
        """
        r = requests.get(
            f"{self.base_url}/workingorders/{deal_id}", headers=self.headers
        )
        r.raise_for_status()
        return r.json()

    # ----------------------------------
    # UPDATE WORKING ORDER
    # ----------------------------------
    def update_working_order(
        self,
        deal_id: str,
        level: float = None,
        good_till_date: str = None,
        guaranteed_stop: bool = None,
        trailing_stop: bool = None,
        stop_level: float = None,
        stop_distance: float = None,
        stop_amount: float = None,
        profit_level: float = None,
        profit_distance: float = None,
        profit_amount: float = None,
    ) -> dict:
        """
        Update a limit or stop working order.

        """

        # Validate parameter combinations
        if guaranteed_stop is not None and trailing_stop is not None:
            if guaranteed_stop and trailing_stop:
                raise ValueError(
                    "Cannot set both guaranteedStop and trailingStop to True"
                )

        if guaranteed_stop and guaranteed_stop is True:
            if not any([stop_level, stop_distance, stop_amount]):
                raise ValueError(
                    "If guaranteedStop=True, must provide stopLevel, stopDistance, or stopAmount"
                )

        if trailing_stop and trailing_stop is True:
            if stop_distance is None:
                raise ValueError("If trailingStop=True, must provide stopDistance")

        # Validate good_till_date format if provided
        if good_till_date:
            try:
                datetime.fromisoformat(good_till_date.replace("Z", ""))
            except ValueError:
                raise ValueError("good_till_date must be in format YYYY-MM-DDTHH:MM:SS")

        # Prepare request body
        body = {}

        # Add parameters if provided
        if level is not None:
            if level <= 0:
                raise ValueError("level must be greater than 0")
            body["level"] = level

        if good_till_date is not None:
            body["goodTillDate"] = good_till_date

        if guaranteed_stop is not None:
            body["guaranteedStop"] = guaranteed_stop

        if trailing_stop is not None:
            body["trailingStop"] = trailing_stop

        if stop_level is not None:
            body["stopLevel"] = stop_level

        if stop_distance is not None:
            body["stopDistance"] = stop_distance

        if stop_amount is not None:
            body["stopAmount"] = stop_amount

        if profit_level is not None:
            body["profitLevel"] = profit_level

        if profit_distance is not None:
            body["profitDistance"] = profit_distance

        if profit_amount is not None:
            body["profitAmount"] = profit_amount

        # Make the API request
        r = requests.put(
            f"{self.base_url}/workingorders/{deal_id}", headers=self.headers, json=body
        )
        r.raise_for_status()

        response = r.json()
        print(f"✅ Working order updated: Deal ID {deal_id}")

        return response

    # ----------------------------------
    # DELETE WORKING ORDER
    # ----------------------------------
    def delete_working_order(self, deal_id: str) -> dict:
        """
        Delete (cancel) a working order.
        """
        r = requests.delete(
            f"{self.base_url}/workingorders/{deal_id}", headers=self.headers
        )
        r.raise_for_status()

        response = r.json()
        print(f"✅ Working order deleted: Deal ID {deal_id}")

        return response

    # ----------------------------------
    # WEBSOCKET OHLC STREAM (UPDATED)
    # ----------------------------------
    def stream_ohlc(
        self,
        epics: list[str],
        resolution: str = "MINUTE_15",
        candle_type: str = "classic",
        on_candle_close=None,
        auto_reconnect: bool = True,
        reconnect_delay: int = 5,
    ):
        """Enhanced WebSocket streaming with session management."""
        if not on_candle_close:
            raise ValueError("on_candle_close callback is required")

        ws_url = "wss://api-streaming-capital.backend-capital.com/connect"

        # State management
        self._ws_active = False
        self._ws_stop = threading.Event()
        self._ws_instance = None  # ⬅️ NEW: Store WebSocket instance

        # Candle tracking
        last_ts = {}
        last_candle = {}

        def on_message(ws, raw):
            try:
                msg = json.loads(raw)

                # Handle ping response
                if msg.get("destination") == "ping":
                    return

                # Handle subscription response
                if msg.get("destination") == "OHLCMarketData.subscribe":
                    status = msg.get("status")
                    if status == "OK":
                        subscriptions = msg.get("payload", {}).get("subscriptions", {})
                        print(f"✅ Subscription successful: {subscriptions}")
                    else:
                        error_code = msg.get("payload", {}).get("errorCode")
                        print(f"❌ Subscription failed: {error_code}")

                        if error_code == "error.invalid.session.token":
                            print("🔄 Session expired, renewing...")
                            self._renew_session()
                            subscribe_to_ohlc(ws)
                    return

                # Handle OHLC data
                if msg.get("destination") == "ohlc.event":
                    # ⬅️ NEW: Check if we should stop
                    if self._ws_stop.is_set():
                        return

                    p = msg["payload"]
                    epic = p["epic"]
                    ts = p["t"]
                    res = p.get("resolution", "UNKNOWN")

                    # Check if this is a new candle (different timestamp)
                    if epic in last_ts and ts != last_ts[epic]:
                        if epic in last_candle:
                            closed_candle = last_candle[epic].copy()
                            closed_candle["timestamp"] = datetime.fromtimestamp(
                                closed_candle["t"] / 1000
                            )
                            closed_candle["epic"] = epic
                            closed_candle["resolution"] = last_candle[epic].get(
                                "resolution", res
                            )
                            on_candle_close(closed_candle)

                    # Update last values
                    last_ts[epic] = ts
                    last_candle[epic] = p

            except Exception as e:
                if not self._ws_stop.is_set():  # ⬅️ NEW: Only print if not stopping
                    print(f"❌ Error processing message: {e}")
                    import traceback

                    traceback.print_exc()

        def subscribe_to_ohlc(ws):
            """Helper function to subscribe to OHLC data."""
            ws_headers = self.get_websocket_headers()

            sub_msg = {
                "destination": "OHLCMarketData.subscribe",
                "correlationId": str(int(time.time() * 1000)),
                "cst": ws_headers["CST"],
                "securityToken": ws_headers["X-SECURITY-TOKEN"],
                "payload": {
                    "epics": epics,
                    "resolutions": [resolution],
                    "type": candle_type,
                },
            }
            ws.send(json.dumps(sub_msg))
            print(f"📡 Sent subscription for {epics} {resolution}")

        def on_open(ws):
            print("✅ WebSocket connection opened")

            ws_headers = self.get_websocket_headers()

            ping_msg = {
                "destination": "ping",
                "correlationId": str(int(time.time() * 1000)),
                "cst": ws_headers["CST"],
                "securityToken": ws_headers["X-SECURITY-TOKEN"],
            }
            ws.send(json.dumps(ping_msg))

            time.sleep(0.5)
            subscribe_to_ohlc(ws)

            self._ws_active = True

        def on_error(ws, error):
            if (
                not self._ws_stop.is_set()
            ):  # ⬅️ NEW: Only print if not intentionally stopping
                print(f"❌ WebSocket error: {error}")
                if auto_reconnect:
                    print(f"⏳ Reconnecting in {reconnect_delay} seconds...")
                    time.sleep(reconnect_delay)
                    if not self._ws_stop.is_set():  # ⬅️ Check again before reconnecting
                        connect_websocket()

        def on_close(ws, close_status_code, close_msg):
            if (
                not self._ws_stop.is_set()
            ):  # ⬅️ NEW: Only print if not intentionally stopping
                print(f"🔌 WebSocket closed: Code={close_status_code}")

            self._ws_active = False

            if auto_reconnect and not self._ws_stop.is_set():
                print(f"⏳ Reconnecting in {reconnect_delay} seconds...")
                time.sleep(reconnect_delay)
                if not self._ws_stop.is_set():  # ⬅️ Check again
                    connect_websocket()

        def connect_websocket():
            """Connect to WebSocket with current session."""
            if self._ws_stop.is_set():  # ⬅️ NEW: Don't connect if stopping
                return None

            print(f"🌐 Connecting to WebSocket...")

            ws_headers = self.get_websocket_headers()

            ws = websocket.WebSocketApp(
                ws_url,
                on_open=on_open,
                on_message=on_message,
                on_error=on_error,
                on_close=on_close,
                header=[
                    f"X-CAP-API-KEY: {self.api_key}",
                    f"CST: {ws_headers['CST']}",
                    f"X-SECURITY-TOKEN: {ws_headers['X-SECURITY-TOKEN']}",
                ],
            )

            self._ws_instance = ws  # ⬅️ NEW: Store instance

            # Run in background thread
            def run_ws():
                ws.run_forever(
                    ping_interval=30,
                    ping_timeout=10,
                    ping_payload=json.dumps(
                        {
                            "destination": "ping",
                            "correlationId": str(int(time.time() * 1000)),
                        }
                    ),
                    reconnect=5,
                )

            thread = threading.Thread(target=run_ws, daemon=True)
            thread.start()
            return ws

        # Start the WebSocket connection
        return connect_websocket()

    def stop_streaming(self):
        """Stop the WebSocket streaming."""
        print("🛑 Stopping WebSocket stream...")
        self._ws_stop.set()  # Set the stop flag
        self._ws_active = False

        # ⬅️ NEW: Actually close the WebSocket connection
        if hasattr(self, "_ws_instance") and self._ws_instance:
            try:
                self._ws_instance.close()
                print("✅ WebSocket closed successfully")
            except Exception as e:
                print(f"⚠️  Error closing WebSocket: {e}")

        # Give it a moment to fully close
        time.sleep(0.5)


def add_day_levels(
    df: pd.DataFrame,
    session_start_hour: int | None = None,  # ⬅️ set None to disable shifting
) -> pd.DataFrame:
    """
    Adds previous day high/low (bid & ask) to Capital.com candle data.

    session_start_hour:
        2    → trading day starts at 02:00 (Capital.com default)
        None → use calendar day (00:00)
    """
    df = df.copy()

    # -----------------------------
    # Safe bid / ask extraction
    # -----------------------------
    def parse_price(x):
        if isinstance(x, dict):
            return x
        if isinstance(x, str):
            try:
                return ast.literal_eval(x)
            except:
                # Try JSON parsing
                return json.loads(x.replace("'", '"'))
        raise ValueError(f"Unexpected price format: {type(x)}")

    for col in ["openPrice", "highPrice", "lowPrice", "closePrice"]:
        parsed = df[col].apply(parse_price)
        df[f"{col}_bid"] = parsed.apply(lambda x: x["bid"])
        df[f"{col}_ask"] = parsed.apply(lambda x: x["ask"])

    # -----------------------------
    # Trading day logic (optional shift)
    # -----------------------------
    ts = pd.to_datetime(df["timestamp"])

    if session_start_hour is not None:
        ts = ts - pd.Timedelta(hours=session_start_hour)

    df["trading_day"] = ts.dt.date

    # -----------------------------
    # Previous day levels
    # -----------------------------
    daily = (
        df.groupby("trading_day")
        .agg(
            prev_high_bid=("highPrice_bid", "max"),
            prev_high_ask=("highPrice_ask", "max"),
            prev_low_bid=("lowPrice_bid", "min"),
            prev_low_ask=("lowPrice_ask", "min"),
        )
        .reset_index()
    )

    return daily

In [ ]:
### ====================================== ###
### Save yesterday's GOLD levels to CSV   ###
### ====================================== ###
if __name__ == "__main__":
    client = CapitalClient(
        api_key=os.getenv("CAPITAL_DEMO_API_KEY"),
        identifier=os.getenv("CAPITAL_IDENTIFIER"),
        password=os.getenv("CAPITAL_PASSWORD"),
    )

    client.login()

    # Define yesterday in UTC explicitly
    yesterday_start = (datetime.now(timezone.utc) - timedelta(days=1)).replace(
        hour=0, minute=0, second=0, microsecond=0
    )
    yesterday_end = yesterday_start.replace(hour=23, minute=59, second=59)

    # Keep them timezone-aware for clarity
    from_date = yesterday_start.isoformat()
    to_date = yesterday_end.isoformat()

    print(f"Fetching data from {from_date} to {to_date}")

    yesterday_data_df = client.get_historical_prices(
        epic="GOLD",
        resolution="MINUTE_15",
        from_date=from_date,
        to_date=to_date,
    )

    # Filter to ensure we only have yesterday's data
    yesterday_data_df["timestamp"] = pd.to_datetime(yesterday_data_df["timestamp"])

    # Filter out any candles from the next day
    yesterday_data_df = yesterday_data_df[
        yesterday_data_df["timestamp"].dt.date == yesterday_start.date()
    ].copy()

    print(f"\nTotal candles after filtering: {len(yesterday_data_df)}")
    print(
        f"Date range: {yesterday_data_df['timestamp'].min()} to {yesterday_data_df['timestamp'].max()}"
    )

    if len(yesterday_data_df) > 0:
        yesterdays_label = add_day_levels(yesterday_data_df)

        # read csv and append new data, create if doesn't exist
        try:
            gold_yesterday_levels_file_df = pd.read_csv("gold_yesterday_levels.csv")
        except FileNotFoundError:
            gold_yesterday_levels_file_df = pd.DataFrame()

        # append new data
        gold_yesterday_levels_file_df = pd.concat(
            [gold_yesterday_levels_file_df, yesterdays_label], ignore_index=True
        )

        # save to csv
        gold_yesterday_levels_file_df.to_csv("gold_yesterday_levels.csv", index=False)

        print(
            f"\n✅ Saved {len(yesterdays_label)} day(s) of levels to gold_yesterday_levels.csv"
        )
    else:
        print("\n⚠️  No data for yesterday")

✅ Logged in successfully
Fetching data from 2026-01-29T00:00:00+00:00 to 2026-01-29T23:59:59+00:00

📥 Downloading GOLD MINUTE_15 candles
   Date range: 2026-01-29 00:00:00 to 2026-01-29 23:59:59
  2026-01-29 00:00:00 → 2026-01-29 23:59:59
✅ Done: 92 candles (2026-01-29 01:00:00 → 2026-01-30 00:45:00)
⚠️  Missing 4 candles at: [Timestamp('2026-01-29 23:15:00'), Timestamp('2026-01-29 23:45:00'), Timestamp('2026-01-29 23:00:00'), Timestamp('2026-01-29 23:30:00')]...

Total candles after filtering: 88
Date range: 2026-01-29 01:00:00 to 2026-01-29 22:45:00

✅ Saved 1 day(s) of levels to gold_yesterday_levels.csv


In [ ]:
### ====================================== ###
###        LIVE STREAMING MAIN LOOP        ###
### ====================================== ###
if __name__ == "__main__":
    # Initialize client
    client = CapitalClient(
        api_key=os.getenv("CAPITAL_DEMO_API_KEY"),
        identifier=os.getenv("CAPITAL_IDENTIFIER"),
        password=os.getenv("CAPITAL_PASSWORD"),
    )

    # Login initially
    client.login()

    def on_candle_closed(candle):
        print("\n" + "=" * 60)
        print(f"🕯️ CLOSED CANDLE: {candle['epic']}")
        print(f"📅 Time: {candle['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}")

        # Use single-letter keys from Capital.com's payload
        print(
            f"📊 O:{candle['o']:.5f} H:{candle['h']:.5f} L:{candle['l']:.5f} C:{candle['c']:.5f}"
        )
        print(f"🔧 Resolution: {candle.get('resolution', 'N/A')}")
        print("=" * 60)

    # Search for markets
    try:
        markets = client.search_markets("gold")

        if markets:
            epic_to_use = markets[0]["epic"]
            print(f"\n✅ Using epic: {epic_to_use}")
        else:
            epic_to_use = "CS.D.EURUSD.MINI.IP"  # Fallback
            print(f"\n⚠️  No markets found, using: {epic_to_use}")

    except Exception as e:
        print(f"❌ Could not fetch markets: {e}")
        epic_to_use = "CS.D.EURUSD.MINI.IP"  # Default fallback

    # Start WebSocket
    print(f"\n🚀 Starting WebSocket for: {epic_to_use}")
    ws = client.stream_ohlc(
        epics=[epic_to_use],
        resolution="MINUTE",
        on_candle_close=on_candle_closed,
        auto_reconnect=True,
        reconnect_delay=5,
    )

    # Keep main thread alive
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("\n👋 Stopping WebSocket...")
        client.stop_streaming()

✅ Logged in successfully

✅ Using epic: GOLD

🚀 Starting WebSocket for: GOLD
🌐 Connecting to WebSocket...
✅ WebSocket connection opened
📡 Sent subscription for ['GOLD'] MINUTE
✅ Subscription successful: {'GOLD:MINUTE:classic': 'PROCESSED'}

🕯️ CLOSED CANDLE: GOLD
📅 Time: 2026-01-30 12:03:00
📊 O:5164.36000 H:5164.36000 L:5153.06000 C:5155.31000
🔧 Resolution: MINUTE

🕯️ CLOSED CANDLE: GOLD
📅 Time: 2026-01-30 12:04:00
📊 O:5155.58000 H:5163.68000 L:5155.58000 C:5160.54000
🔧 Resolution: MINUTE

🕯️ CLOSED CANDLE: GOLD
📅 Time: 2026-01-30 12:05:00
📊 O:5160.46000 H:5169.05000 L:5153.97000 C:5165.78000
🔧 Resolution: MINUTE

🕯️ CLOSED CANDLE: GOLD
📅 Time: 2026-01-30 12:06:00
📊 O:5165.75000 H:5174.99000 L:5165.75000 C:5173.23000
🔧 Resolution: MINUTE

🕯️ CLOSED CANDLE: GOLD
📅 Time: 2026-01-30 12:07:00
📊 O:5173.35000 H:5180.04000 L:5172.29000 C:5178.41000
🔧 Resolution: MINUTE

🕯️ CLOSED CANDLE: GOLD
📅 Time: 2026-01-30 12:08:00
📊 O:5178.34000 H:5179.97000 L:5174.25000 C:5174.59000
🔧 Resolution: MINUT

In [36]:
from dataclasses import dataclass
from datetime import date


# ====================================================
# 🕯️ BID / ASK CANDLE
# ====================================================
@dataclass
class Candle:
    timestamp: object

    open_bid: float
    open_ask: float

    high_bid: float
    high_ask: float

    low_bid: float
    low_ask: float

    close_bid: float
    close_ask: float


# ====================================================
# 🚀 LIVE STRATEGY: YESTERDAY HIGH/LOW BREAK
# ====================================================
class LiveYesterdayHighLowTrader:
    def __init__(
        self,
        symbol: str,
        risk_per_trade: float,
        tp_pips: float,
        symbol_info: dict,
        rr: float = 2.0,  # Risk:Reward ratio
    ):
        self.symbol = symbol
        self.risk_per_trade = risk_per_trade
        self.tp_pips = tp_pips
        self.info = symbol_info
        self.rr = rr

        # DAILY STATE
        self.today = None
        self.traded_today = False

        # STRATEGY STATE
        self.y_high = None
        self.y_low = None
        self.break_candle = None
        self.direction = None
        self.prev_candle = None

    # ------------------------------------------------
    # 🔄 DAILY RESET (BID LEVELS)
    # ------------------------------------------------
    def set_new_day(self, today_date: date, yesterday_high_bid, yesterday_low_bid):
        self.today = today_date
        self.y_high = yesterday_high_bid
        self.y_low = yesterday_low_bid

        self.traded_today = False
        self.break_candle = None
        self.direction = None
        self.prev_candle = None

    # ------------------------------------------------
    # 🔥 MAIN ENTRY POINT: new candle arrives
    # ------------------------------------------------
    def on_new_candle(self, candle: Candle, capital: float):

        if self.traded_today:
            self.prev_candle = candle
            return None

        if self.prev_candle is None:
            self.prev_candle = candle
            return None

        # ------------------ C1 BREAK (BID) ------------------
        if self.break_candle is None:

            if (
                self.prev_candle.open_bid <= self.y_high
                and self.prev_candle.close_bid > self.y_high
            ):
                self.break_candle = self.prev_candle
                self.direction = "long"

            elif (
                self.prev_candle.open_bid >= self.y_low
                and self.prev_candle.close_bid < self.y_low
            ):
                self.break_candle = self.prev_candle
                self.direction = "short"

            self.prev_candle = candle
            return None

        # ------------------ C2 CONFIRM ------------------
        if self.direction == "long" and candle.close_bid <= self.break_candle.close_bid:
            self._invalidate()
            self.prev_candle = candle
            return None

        if (
            self.direction == "short"
            and candle.close_bid >= self.break_candle.close_bid
        ):
            self._invalidate()
            self.prev_candle = candle
            return None

        # ------------------ C3 ENTRY ------------------
        trade = self._create_trade(candle, capital)
        self.traded_today = True
        self._invalidate()
        self.prev_candle = candle

        return trade

    # ------------------------------------------------
    # 🧮 LOT SIZE & DYNAMIC TP
    # ------------------------------------------------
    def _create_trade(self, candle: Candle, capital: float):
        if self.direction == "long":
            entry = candle.open_ask
            sl = self.break_candle.low_bid
        else:
            entry = candle.open_bid
            sl = self.break_candle.high_ask

        # 1️⃣ Stop loss distance
        sl_distance = abs(entry - sl)

        # 2️⃣ Risk amount
        risk_amount = capital * self.risk_per_trade

        # 3️⃣ Lot calculation
        ticks = sl_distance / self.info["tick_size"]
        loss_per_lot = ticks * self.info["tick_value"]
        raw_lot = risk_amount / loss_per_lot
        step = self.info["lot_step"]
        min_lot = self.info["min_lot"]
        lot = max(round(int(raw_lot / step) * step, 3), min_lot)

        # 4️⃣ TP distance based on RR
        tp_distance = self.rr * sl_distance

        if self.direction == "long":
            tp = entry + tp_distance
        else:
            tp = entry - tp_distance

        # 5️⃣ Optional: dynamic pip_size
        dynamic_pip_size = tp_distance / self.tp_pips

        return {
            "symbol": self.symbol,
            "direction": self.direction,
            "lot": lot,
            "entry_price": entry,
            "stop_loss": sl,
            "take_profit": tp,
            "tp_distance": tp_distance,
            "dynamic_pip_size": dynamic_pip_size,
            "entry_time": candle.timestamp,
        }

    # ------------------------------------------------
    # ⚡ INTERNAL RESET
    # ------------------------------------------------
    def _invalidate(self):
        self.break_candle = None
        self.direction = None

In [1]:
import os
import json
import time
import pandas as pd
from datetime import datetime, date, timedelta, timezone
from dataclasses import dataclass
from typing import Optional, Dict, Tuple
import threading
import requests


# ----------------------------------------------------
# DATA CLASSES
# ----------------------------------------------------
@dataclass
class Candle:
    timestamp: datetime
    open: float
    high: float
    low: float
    close: float


class GoldYesterdayHighLowTrader:
    def __init__(
        self,
        epic: str = "CS.D.USCGC.MINI.IP",  # Gold CFD epic on Capital.com
        risk_percent: float = 2.0,  # 2% risk per trade
        tp_pips: float = 300,  # 300 pips TP (3.00 USD for Gold)
        account_balance: float = 10000,
    ):
        self.epic = epic
        self.risk_percent = risk_percent / 100  # Convert to decimal
        self.tp_pips = tp_pips
        self.account_balance = account_balance

        # Gold specific settings (CFD on Capital.com)
        self.pip_size = 0.01  # For Gold, 1 pip = $0.01
        self.min_lot = 0.01  # Minimum lot size
        self.lot_step = 0.01  # Lot increment
        self.tick_size = 0.01  # Minimum price movement
        self.contract_size = 100  # Ounces per contract (CFD)
        self.tick_value = 1.0  # $1 per tick (0.01 move)

        # DAILY STATE
        self.today = None
        self.traded_today = False

        # STRATEGY STATE
        self.y_high = None
        self.y_low = None
        self.break_candle = None
        self.direction = None
        self.prev_candle = None

        # TRACKING
        self.trades_today = []
        self.active_trade = None
        self.position_id = None

    # ----------------------------------------------------
    # 🔄 DAILY RESET
    # ----------------------------------------------------
    def set_new_day(self, today_date: date, yesterday_high, yesterday_low):
        self.today = today_date
        self.y_high = yesterday_high
        self.y_low = yesterday_low

        self.traded_today = False
        self.break_candle = None
        self.direction = None
        self.prev_candle = None
        self.active_trade = None
        self.position_id = None

        print(f"\n📅 NEW GOLD TRADING DAY: {today_date}")
        print(f"📊 Yesterday H: ${yesterday_high:.2f} | L: ${yesterday_low:.2f}")
        print(f"📈 Range: ${yesterday_high - yesterday_low:.2f}")

    # ----------------------------------------------------
    # 🔥 MAIN ENTRY POINT (CALL ON EVERY NEW CANDLE)
    # ----------------------------------------------------
    def on_new_candle(self, candle: Candle) -> Optional[Dict]:
        candle_date = candle.timestamp.date()

        # Check if it's a new day
        if self.today != candle_date:
            print(f"\n🔄 New trading day detected: {candle_date}")
            return None

        if self.traded_today:
            self.prev_candle = candle
            return None

        # Need previous candle
        if self.prev_candle is None:
            self.prev_candle = candle
            return None

        # ------------------ C1 BREAK ------------------
        if self.break_candle is None:
            # Check for LONG breakout (close above yesterday's high)
            if (
                self.prev_candle.open <= self.y_high
                and self.prev_candle.close > self.y_high
            ):
                self.break_candle = self.prev_candle
                self.direction = "BUY"
                print(f"\n📈 GOLD LONG BREAKOUT detected!")
                print(f"   Time: {self.prev_candle.timestamp.strftime('%H:%M:%S')}")
                print(
                    f"   Candle: O=${self.prev_candle.open:.2f}, C=${self.prev_candle.close:.2f}"
                )
                print(f"   Break Level: ${self.y_high:.2f}")
                print(f"   Break Size: ${self.prev_candle.close - self.y_high:.2f}")

            # Check for SHORT breakout (close below yesterday's low)
            elif (
                self.prev_candle.open >= self.y_low
                and self.prev_candle.close < self.y_low
            ):
                self.break_candle = self.prev_candle
                self.direction = "SELL"
                print(f"\n📉 GOLD SHORT BREAKOUT detected!")
                print(f"   Time: {self.prev_candle.timestamp.strftime('%H:%M:%S')}")
                print(
                    f"   Candle: O=${self.prev_candle.open:.2f}, C=${self.prev_candle.close:.2f}"
                )
                print(f"   Break Level: ${self.y_low:.2f}")
                print(f"   Break Size: ${self.y_low - self.prev_candle.close:.2f}")

            self.prev_candle = candle
            return None

        # ------------------ C2 CONFIRM ------------------
        if candle.timestamp == self.break_candle.timestamp:
            return None

        # LONG: Check if current close is below break candle close (invalidation)
        if self.direction == "BUY" and candle.close <= self.break_candle.close:
            print(
                f"❌ LONG invalidated - Price closed below breakout candle at ${candle.close:.2f}"
            )
            self._invalidate()
            self.prev_candle = candle
            return None

        # SHORT: Check if current close is above break candle close (invalidation)
        if self.direction == "SELL" and candle.close >= self.break_candle.close:
            print(
                f"❌ SHORT invalidated - Price closed above breakout candle at ${candle.close:.2f}"
            )
            self._invalidate()
            self.prev_candle = candle
            return None

        # ------------------ C3 ENTRY ------------------
        trade = self._create_trade(candle)
        if trade and trade["quantity"] > 0:
            self.traded_today = True
            self.active_trade = trade
            self.trades_today.append(trade)

            print(f"\n{'='*60}")
            print(f"✅ GOLD TRADE SIGNAL GENERATED!")
            print(f"   Direction: {trade['direction']}")
            print(f"   Entry: ${trade['entry_price']:.2f}")
            print(f"   Stop Loss: ${trade['stop_loss']:.2f}")
            print(f"   Take Profit: ${trade['take_profit']:.2f}")
            print(f"   Quantity: {trade['quantity']:.2f} oz")
            print(f"   Risk: ${trade['risk_amount']:.2f}")
            print(f"   Potential Reward: ${trade['potential_reward']:.2f}")
            print(f"   Risk/Reward: 1:{trade['risk_reward_ratio']:.2f}")
            print(f"   Time: {trade['entry_time'].strftime('%Y-%m-%d %H:%M:%S')}")
            print("=" * 60)

            self._invalidate()
            self.prev_candle = candle
            return trade

        self._invalidate()
        self.prev_candle = candle
        return None

    # ----------------------------------------------------
    # 🧮 DYNAMIC LOT SIZE CALCULATION FOR GOLD
    # ----------------------------------------------------
    def _calculate_position_size(self, entry: float, sl: float) -> Tuple[float, Dict]:
        """Calculate position size based on risk and stop loss distance"""

        # Risk amount in USD
        risk_amount = self.account_balance * self.risk_percent

        # Stop loss distance in dollars
        sl_distance = abs(entry - sl)

        if sl_distance <= 0:
            return 0.0, {}

        # For Gold CFD on Capital.com:
        # 1 oz of Gold = contract_size * price
        # Risk per oz = sl_distance * contract_size

        # Calculate raw quantity (in oz)
        risk_per_oz = sl_distance * self.contract_size
        raw_quantity = risk_amount / risk_per_oz if risk_per_oz > 0 else 0

        # Apply broker constraints
        quantity = max(self.min_lot, raw_quantity)
        quantity = round(quantity / self.lot_step) * self.lot_step

        # Calculate position value
        position_value = quantity * entry

        # Calculate potential reward
        tp_distance = self.tp_pips * self.pip_size
        potential_reward = tp_distance * self.contract_size * quantity
        risk_reward_ratio = tp_distance / sl_distance if sl_distance > 0 else 0

        return quantity, {
            "risk_amount": risk_amount,
            "sl_distance": sl_distance,
            "position_value": position_value,
            "potential_reward": potential_reward,
            "risk_reward_ratio": risk_reward_ratio,
        }

    # ----------------------------------------------------
    # 📦 CREATE TRADE OBJECT
    # ----------------------------------------------------
    def _create_trade(self, candle: Candle) -> Optional[Dict]:
        entry = candle.open
        current_price = candle.close

        if self.direction == "BUY":
            sl = self.break_candle.low
            tp = entry + (self.tp_pips * self.pip_size)
        else:  # SELL
            sl = self.break_candle.high
            tp = entry - (self.tp_pips * self.pip_size)

        # Calculate position size
        quantity, calc_info = self._calculate_position_size(entry, sl)

        if quantity <= 0:
            print(f"⚠️  Position size calculation failed or too small")
            return None

        # Check if SL is too close (less than 50 pips for Gold)
        sl_pips = abs(entry - sl) / self.pip_size
        if sl_pips < 50:  # Minimum 50 pips stop loss for Gold
            print(f"⚠️  Stop loss too close: {sl_pips:.0f} pips (min 50)")
            return None

        trade = {
            "epic": self.epic,
            "direction": self.direction,
            "quantity": quantity,
            "entry_price": entry,
            "current_price": current_price,
            "stop_loss": sl,
            "take_profit": tp,
            "entry_time": candle.timestamp,
            "status": "PENDING",
            "sl_pips": sl_pips,
            "tp_pips": self.tp_pips,
            "risk_amount": calc_info["risk_amount"],
            "position_value": calc_info["position_value"],
            "potential_reward": calc_info["potential_reward"],
            "risk_reward_ratio": calc_info["risk_reward_ratio"],
        }

        return trade

    def _invalidate(self):
        self.break_candle = None
        self.direction = None

    # ----------------------------------------------------
    # 📊 TRADE MONITORING
    # ----------------------------------------------------
    def check_trade_status(self, current_price: float) -> Optional[Dict]:
        """Check if active trade has hit TP or SL"""
        if not self.active_trade:
            return None

        trade = self.active_trade
        entry = trade["entry_price"]
        sl = trade["stop_loss"]
        tp = trade["take_profit"]
        direction = trade["direction"]

        # Calculate current P&L
        if direction == "BUY":
            pnl = (current_price - entry) * self.contract_size * trade["quantity"]
            pnl_pips = (current_price - entry) / self.pip_size
        else:  # SELL
            pnl = (entry - current_price) * self.contract_size * trade["quantity"]
            pnl_pips = (entry - current_price) / self.pip_size

        trade["current_pnl"] = pnl
        trade["current_pnl_pips"] = pnl_pips

        # Check exit conditions
        if direction == "BUY":
            if current_price <= sl:
                return {
                    "action": "CLOSE",
                    "reason": "STOP_LOSS",
                    "price": sl,
                    "pnl": (sl - entry) * self.contract_size * trade["quantity"],
                    "trade": trade,
                }
            elif current_price >= tp:
                return {
                    "action": "CLOSE",
                    "reason": "TAKE_PROFIT",
                    "price": tp,
                    "pnl": (tp - entry) * self.contract_size * trade["quantity"],
                    "trade": trade,
                }
        else:  # SELL
            if current_price >= sl:
                return {
                    "action": "CLOSE",
                    "reason": "STOP_LOSS",
                    "price": sl,
                    "pnl": (entry - sl) * self.contract_size * trade["quantity"],
                    "trade": trade,
                }
            elif current_price <= tp:
                return {
                    "action": "CLOSE",
                    "reason": "TAKE_PROFIT",
                    "price": tp,
                    "pnl": (entry - tp) * self.contract_size * trade["quantity"],
                    "trade": trade,
                }

        # Return current status
        return {
            "action": "HOLD",
            "price": current_price,
            "pnl": pnl,
            "pnl_pips": pnl_pips,
            "trade": trade,
        }


# ----------------------------------------------------
# YESTERDAY LEVELS MANAGER FOR GOLD
# ----------------------------------------------------
class GoldLevelsManager:
    def __init__(self, client: CapitalClient):
        self.client = client
        self.levels_file = "gold_yesterday_levels.json"
        self.gold_epic = "CS.D.USCGC.MINI.IP"  # Gold CFD epic

    def calculate_yesterday_levels(self) -> Dict:
        """Calculate yesterday's high and low for Gold"""
        # Get yesterday's date (UTC)
        utc_now = datetime.now(timezone.utc)
        yesterday_start = (utc_now - timedelta(days=2)).replace(
            hour=0, minute=0, second=0, microsecond=0
        )

        yesterday_end = yesterday_start + timedelta(days=1) - timedelta(seconds=1)

        # Convert to ISO format (remove timezone for Capital API)
        from_date = yesterday_start.replace(tzinfo=None).isoformat()
        to_date = yesterday_end.replace(tzinfo=None).isoformat()

        print(f"\n📊 Fetching yesterday's GOLD data...")
        print(f"   From: {from_date}")
        print(f"   To: {to_date}")

        try:
            # Get daily candles for yesterday
            df = self.client.get_historical_prices(
                epic=self.gold_epic,
                resolution="DAY",
                from_date=from_date,
                to_date=to_date,
            )

            if df.empty:
                print("❌ No GOLD data received for yesterday")
                return {}

            # Get high and low
            y_high = df["highPrice"].max()
            y_low = df["lowPrice"].min()

            result = {
                "epic": self.gold_epic,
                "date": yesterday_start.date().isoformat(),
                "yesterday_high": float(y_high),
                "yesterday_low": float(y_low),
                "range": float(y_high - y_low),
                "calculated_at": utc_now.isoformat(),
                "candles_count": len(df),
            }

            print(f"\n✅ GOLD Yesterday's Levels:")
            print(f"   High: ${y_high:.2f}")
            print(f"   Low: ${y_low:.2f}")
            print(f"   Range: ${y_high - y_low:.2f}")
            print(f"   Date: {yesterday_start.date()}")

            # Save to file
            self._save_levels(result)

            return result

        except Exception as e:
            print(f"❌ Error calculating yesterday levels: {e}")
            return {}

    def load_yesterday_levels(self) -> Dict:
        """Load yesterday's levels from JSON file"""
        try:
            with open(self.levels_file, "r") as f:
                data = json.load(f)

            print(f"✅ Loaded GOLD levels from {self.levels_file}")
            print(f"   Date: {data['date']}")
            print(f"   High: ${data['yesterday_high']:.2f}")
            print(f"   Low: ${data['yesterday_low']:.2f}")

            return data
        except FileNotFoundError:
            print(f"⚠️  {self.levels_file} not found.")
            return {}
        except json.JSONDecodeError:
            print(f"⚠️  {self.levels_file} is corrupted.")
            return {}

    def _save_levels(self, levels: Dict):
        """Save levels to JSON file"""
        with open(self.levels_file, "w") as f:
            json.dump(levels, f, indent=2)
        print(f"💾 Saved GOLD levels to {self.levels_file}")


# ----------------------------------------------------
# LIVE GOLD TRADING BOT
# ----------------------------------------------------
class LiveGoldTradingBot:
    def __init__(self, client: CapitalClient, account_balance: float = 10000):
        self.client = client
        self.account_balance = account_balance

        # Initialize managers
        self.levels_manager = GoldLevelsManager(client)

        # Initialize trader
        self.trader = GoldYesterdayHighLowTrader(
            account_balance=account_balance,
            risk_percent=2.0,  # 2% risk
            tp_pips=300,  # 300 pips = $3.00
        )

        # Trading state
        self.is_running = False
        self.last_candle_time = None
        self.current_price = None

    def initialize_day(self) -> bool:
        """Initialize trading for the current day"""
        today = datetime.now(timezone.utc).date()

        print(f"\n{'='*60}")
        print(f"🎯 INITIALIZING GOLD TRADING BOT")
        print(f"💰 Account Balance: ${self.account_balance:,.2f}")
        print(f"📅 Today's Date: {today}")
        print("=" * 60)

        # Try to load existing levels first
        levels = self.levels_manager.load_yesterday_levels()

        # If no levels or levels are from wrong day, calculate new ones
        if not levels or levels.get("date") != (today - timedelta(days=1)).isoformat():
            print("\n🔄 Calculating fresh yesterday levels...")
            levels = self.levels_manager.calculate_yesterday_levels()
            if not levels:
                print("❌ Failed to get yesterday's levels")
                return False

        # Initialize trader
        self.trader.set_new_day(
            today_date=today,
            yesterday_high=levels["yesterday_high"],
            yesterday_low=levels["yesterday_low"],
        )

        # Update trader's account balance
        self.trader.account_balance = self.account_balance

        print(f"\n✅ GOLD Trading Bot Initialized!")
        print(f"   Symbol: {self.trader.epic}")
        print(f"   Risk per trade: {self.trader.risk_percent*100}%")
        print(f"   Take profit: {self.trader.tp_pips} pips")
        print(f"   Min lot size: {self.trader.min_lot}")

        return True

    def on_candle_close(self, candle_data: Dict):
        """Process each closed candle"""
        try:
            # Create Candle object
            candle = Candle(
                timestamp=candle_data["timestamp"],
                open=candle_data["o"],
                high=candle_data["h"],
                low=candle_data["l"],
                close=candle_data["c"],
            )

            # Store current price for monitoring
            self.current_price = candle.close

            # Check if this is a new candle (not duplicate)
            if self.last_candle_time and candle.timestamp == self.last_candle_time:
                return

            self.last_candle_time = candle.timestamp

            # Check if we need to initialize new day
            if not self.trader.today:
                self.initialize_day()

            # Get trade signal
            trade_signal = self.trader.on_new_candle(candle)

            if trade_signal:
                # Execute trade on Capital.com
                self._execute_trade(trade_signal)

            # Monitor active trade
            if self.trader.active_trade:
                status = self.trader.check_trade_status(self.current_price)
                if status and status["action"] != "HOLD":
                    self._handle_trade_exit(status)

        except Exception as e:
            print(f"❌ Error processing candle: {e}")
            import traceback

            traceback.print_exc()

    def _execute_trade(self, trade: Dict):
        """Execute trade on Capital.com"""
        print(f"\n🚀 PLACING ORDER ON CAPITAL.COM")
        print(f"   Direction: {trade['direction']}")
        print(f"   Quantity: {trade['quantity']:.2f} oz")
        print(f"   Entry: ${trade['entry_price']:.2f}")

        # Prepare order parameters for Capital.com
        order_data = {
            "epic": trade["epic"],
            "expiry": "-",  # Good till cancelled
            "direction": trade["direction"],
            "size": trade["quantity"],
            "orderType": "MARKET",  # or "LIMIT" if you want limit order
            "timeInForce": "EXECUTE_AND_ELIMINATE",
            "level": trade["entry_price"],
            "guaranteedStop": False,
            "stopDistance": abs(trade["entry_price"] - trade["stop_loss"]),
            "profitDistance": abs(trade["entry_price"] - trade["take_profit"]),
        }

        try:
            # Uncomment when ready to trade
            # response = self.client.place_order(order_data)
            # trade["position_id"] = response.get("dealReference")
            # trade["status"] = "OPEN"
            # self.trader.position_id = trade["position_id"]

            print(f"✅ ORDER PLACED (SIMULATED)")
            print(f"   Position ID: SIM-{int(time.time())}")

        except Exception as e:
            print(f"❌ Order placement failed: {e}")
            trade["status"] = "FAILED"

    def _handle_trade_exit(self, exit_info: Dict):
        """Handle trade exit (TP/SL)"""
        reason = exit_info["reason"]
        price = exit_info["price"]
        pnl = exit_info["pnl"]

        print(f"\n{'='*60}")
        print(f"🎯 TRADE EXIT: {reason}")
        print(f"   Exit Price: ${price:.2f}")
        print(f"   P&L: ${pnl:.2f}")
        print(f"   Status: {'PROFIT' if pnl > 0 else 'LOSS'}")
        print("=" * 60)

        # Update account balance
        self.account_balance += pnl
        self.trader.account_balance = self.account_balance

        print(f"💰 New Account Balance: ${self.account_balance:,.2f}")

        # Close position on Capital.com
        try:
            # Uncomment when ready
            # if self.trader.position_id:
            #     self.client.close_position(self.trader.position_id, trade["quantity"])
            pass
        except Exception as e:
            print(f"❌ Error closing position: {e}")

        # Reset trader state for next day
        self.trader.active_trade = None
        self.trader.position_id = None

    def start_trading(self, resolution: str = "MINUTE_15"):
        """Start the trading bot"""
        print(f"\n🚀 STARTING GOLD TRADING BOT")
        print(f"📏 Resolution: {resolution}")

        # Initialize day
        if not self.initialize_day():
            print("❌ Failed to initialize trading bot")
            return

        self.is_running = True

        # Define candle callback
        def on_candle_closed(candle):
            self.on_candle_close(candle)

        # Start WebSocket stream
        print(f"\n📡 Connecting to Gold WebSocket...")
        ws = self.client.stream_ohlc(
            epics=[self.trader.epic],
            resolution=resolution,
            on_candle_close=on_candle_closed,
        )

        # Keep main thread alive
        try:
            while self.is_running:
                time.sleep(1)

                # Periodic status update
                if self.trader.active_trade and self.current_price:
                    status = self.trader.check_trade_status(self.current_price)
                    if status and status["action"] == "HOLD":
                        # Print P&L update every 30 seconds
                        if int(time.time()) % 30 == 0:
                            print(
                                f"📊 Active Trade: P&L ${status['pnl']:.2f} ({status['pnl_pips']:.0f} pips)"
                            )

        except KeyboardInterrupt:
            print("\n👋 Stopping Gold Trading Bot...")
            self.is_running = False


# ----------------------------------------------------
# MAIN EXECUTION
# ----------------------------------------------------
if __name__ == "__main__":
    # Initialize client
    client = CapitalClient(
        api_key=os.getenv("CAPITAL_DEMO_API_KEY"),
        identifier=os.getenv("CAPITAL_IDENTIFIER"),
        password=os.getenv("CAPITAL_PASSWORD"),
    )

    client.login()

    # Create and start Gold trading bot
    bot = LiveGoldTradingBot(client=client, account_balance=10000)  # Starting balance

    # Start trading (using 15-minute candles)
    bot.start_trading(resolution="MINUTE_15")

NameError: name 'CapitalClient' is not defined